In [1]:
import pandas as pd 
import numpy as np

TWEET_DATA = pd.read_csv("dataset_final_fixing.csv", encoding = "ISO-8859-1")

TWEET_DATA.head()

,tweet,label
0,@LisaAmartatara3 @fadlizon Yg dibangga banggai...,1
1,RT @RustamIbrahim: POLLING: Dari 4 gubernur di...,1
2,ojok ampe Anies Baswedan menikmati hsil proyek...,1
3,RT @ghanieierfan: Pertemuan sangat mengharukan...,0
4,RT @SebastianTeza: jangn nympek Anies Baswedan...,1


In [2]:
# ------ Case Folding --------
# gunakan fungsi Series.str.lower() pada Pandas
TWEET_DATA['tweet'] = TWEET_DATA['tweet'].str.lower()


print('Case Folding Result : \n')
print(TWEET_DATA['tweet'].head(5))
print('\n\n\n')

Case Folding Result : 

0    @lisaamartatara3 @fadlizon yg dibangga banggai...
1    rt @rustamibrahim: polling: dari 4 gubernur di...
2    ojok ampe anies baswedan menikmati hsil proyek...
3    rt @ghanieierfan: pertemuan sangat mengharukan...
4    rt @sebastianteza: jangn nympek anies baswedan...
Name: tweet, dtype: object






In [3]:
import string 
import re #regex library

# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist

# ------ Tokenizing ---------
#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

TWEET_DATA['tweet'] = TWEET_DATA['tweet'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

TWEET_DATA['tweet'] = TWEET_DATA['tweet'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

TWEET_DATA['tweet'] = TWEET_DATA['tweet'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

TWEET_DATA['tweet'] = TWEET_DATA['tweet'].apply(remove_whitespace_multiple)


# NLTK word rokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

TWEET_DATA['tweet_tokens'] = TWEET_DATA['tweet'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n') 
print(TWEET_DATA['tweet_tokens'].head())
print('\n\n\n')

Tokenizing Result : 

0    [lisaamartatara, fadlizon, yg, dibangga, bangg...
1    [rt, rustamibrahim, polling, dari, gubernur, d...
2    [ojok, ampe, anies, baswedan, menikmati, hsil,...
3    [rt, ghanieierfan, pertemuan, sangat, mengharu...
4    [rt, sebastianteza, jangn, nympek, anies, basw...
Name: tweet_tokens, dtype: object






In [4]:
# NLTK calc frequency distribution
def freqDist_wrapper(text):
    return FreqDist(text)

TWEET_DATA['tweet_tokens_fdist'] = TWEET_DATA['tweet_tokens'].apply(freqDist_wrapper)

print('Frequency Tokens : \n') 
print(TWEET_DATA['tweet_tokens_fdist'].head().apply(lambda x : x.most_common()))

Frequency Tokens : 

0    [(lisaamartatara, 1), (fadlizon, 1), (yg, 1), ...
1    [(rt, 1), (rustamibrahim, 1), (polling, 1), (d...
2    [(ojok, 1), (ampe, 1), (anies, 1), (baswedan, ...
3    [(rt, 1), (ghanieierfan, 1), (pertemuan, 1), (...
4    [(rt, 1), (sebastianteza, 1), (jangn, 1), (nym...
Name: tweet_tokens_fdist, dtype: object


In [6]:
from nltk.corpus import stopwords

# get stopword indonesia
list_stopwords = stopwords.words('indonesian')

# append additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja'])

list_stopwords = set(list_stopwords)

#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if not word in list_stopwords]

TWEET_DATA['tweet_tokens_WSW'] = TWEET_DATA['tweet_tokens'].apply(stopwords_removal) 


print(TWEET_DATA['tweet_tokens_fdist_WSW'].head())

0    [lisaamartatara, fadlizon, dibangga, banggain,...
1    [rustamibrahim, polling, gubernur, jawa, ridwa...
2    [ojok, ampe, anies, baswedan, menikmati, hsil,...
3    [ghanieierfan, pertemuan, mengharukan, aniesba...
4    [sebastianteza, jangn, nympek, anies, baswedan...
Name: tweet_tokens_fdist_WSW, dtype: object


# Normalisasi

In [23]:
normalizad_word = pd.read_excel("normalisasi.xlsx")

normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

TWEET_DATA['tweet_normalized'] = TWEET_DATA['tweet_tokens_WSW'].apply(normalized_term)

In [25]:
TWEET_DATA['tweet_normalized'].head(10)

0    [lisaamartatara, fadlizon, dibangga, banggakan...
1    [rustamibrahim, polling, gubernur, jawa, ridwa...
2    [jangan, sampai, anies, baswedan, menikmati, h...
3    [ghanieierfan, pertemuan, mengharukan, aniesba...
4    [sebastianteza, jangan, nympek, anies, basweda...
5    [tamastefan, mencapai, anies, baswedan, menikm...
6    [ariestariico, niat, cari, prestasi, ilegal, a...
7    [berbatas, anies, baswedan, menikmati, rakitan...
8    [santorotahir, jangan, nyampe, anies, baswedan...
9    [jangan, sampai, anies, baswedan, menikmati, h...
Name: tweet_normalized, dtype: object

In [27]:
# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter


# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in TWEET_DATA['tweet_normalized']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")


# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

TWEET_DATA['tweet_tokens_stemmed'] = TWEET_DATA['tweet_normalized'].swifter.apply(get_stemmed_term)
print(TWEET_DATA['tweet_tokens_stemmed'])

98242
------------------------
lisaamartatara : lisaamartatara
fadlizon : fadlizon
dibangga : bangga
banggakan : bangga
buzzer : buzzer
cendana : cendana
ya : ya
cuman : cuman
orangn : orangn
almarhum : almarhum
soeharton : soeharton
anies : anies
baswedannyg : baswedannyg
lewathehe : lewathehe
rustamibrahim : rustamibrahim
polling : polling
gubernur : gubernur
jawa : jawa
ridwan : ridwan
kamil : kamil
baswedan : baswedan
ganjar : ganjar
pranowo : pranowo
khofifah : khofifah
indar : indar
parawansa : parawansa
penu : penu
jangan : jangan
sampai : sampai
menikmati : nikmat
hasil : hasil
proyek : proyek
monas : monas
yang : yang
penuh : penuh
kejanggalannya : janggal
usutskandalmonasgate : usutskandalmonasgate
ghanieierfan : ghanieierfan
pertemuan : temu
mengharukan : haru
aniesbaswedan : aniesbaswedan
tsuneo : tsuneo
yamada : yamada
orang : orang
tua : tua
angkat : angkat
goodbener : goodbener
kuliah : kuliah
sophia : sophia
univeu : univeu
sebastianteza : sebastianteza
nympek : nympek


skpdnnbagiu : skpdnnbagiu
sandi : sandi
pimpin : pimpin
adem : adem
ayem : ayem
azanjkwhancurkanindonesia : azanjkwhancurkanindonesia
wahabislokal : wahabislokal
ngakak : ngakak
yooonnkwak : yooonnkwak
wkwk : wkwk
kwakuddudeuddudeudeuddnjika : kwakuddudeuddudeudeuddnjika
ngoceh : ngoceh
ginian : ginian
aniesnnwisnhora : aniesnnwisnhora
dipercoyonnsinetronanuddudeuddudeuddudeudeuddnuu : dipercoyonnsinetronanuddudeuddudeuddudeudeuddnuu
gampang : gampang
tinggal : tinggal
kirim : kirim
suratncoba : suratncoba
ahok : ahok
berharinpnjarakan : berharinpnjarakan
melu : melu
penebangan : tebang
httpstcooqsmvap : httpstcooqsmvap
httpstcowfupsu : httpstcowfupsu
usut : usut
tuntas : tuntas
out : out
nusutskandalmonasgate : nusutskandalmonasgate
httpstcoyqohgau : httpstcoyqohgau
marlinaidha : marlinaidha
meresmikan : resmi
layang : layang
cipendawa : cipendawa
rawa : rawa
panjangnnucapan : panjangnnucapan
terima kasih : terima kasih
walu : walu
hacknet : hacknet
please : please
takut : takut
kejan

brigadepenolong : brigadepenolong
kwardajatim : kwardajatim
nnpramukajatim : nnpramukajatim
gerakanpramuka : gerakanpramuka
jatimguyub : jatimguyub
httpstcohmmymlvdo : httpstcohmmymlvdo
didifathur : didifathur
jauhkan : jauh
negeri : negeri
ekstrim : ekstrim
berdampak : dampak
conanidn : conanidn
de : de
dipikir : pikir
solusinya : solusi
bin : bin
kabur : kabur
disananmasa : disananmasa
apoy : apoy
wali : wali
band : band
ponpesnya : ponpesnya
diterjanguabanjir : diterjanguabanjir
httpstcoxumisatdp : httpstcoxumisatdp
httpstcokaewwdga : httpstcokaewwdga
httpstcoxqvwkecji : httpstcoxqvwkecji
httpstcoszngzzxidt : httpstcoszngzzxidt
aliexsmart : aliexsmart
catat : catat
yannhttpstcolfqbbybkro : yannhttpstcolfqbbybkro
aksi : aksi
pramuka : pramuka
kwarcabproboli : kwarcabproboli
nbrigadepenolong : nbrigadepenolong
httpstcoutpvidgoiv : httpstcoutpvidgoiv
kakekhalal : kakekhalal
meninggal : tinggal
virus : virus
corona : corona
china : china
azabnkorban : azabnkorban
diu : diu
huminitiative

httpstcopyvppcxjx : httpstcopyvppcxjx
bmkg : bmkg
menyebut : sebut
httpstcohbvfxzknj : httpstcohbvfxzknj
ucforum : ucforum
bikers : bikers
bantenud : bantenud
by : by
adiuapro : adiuapro
httpstcobwfjeim : httpstcobwfjeim
httpstcoyzptyomjfz : httpstcoyzptyomjfz
klhk : klhk
langkahlangkah : langkahlangkah
terukur : ukur
mengatasi : atas
tangeranu : tangeranu
world : world
war : war
wildfire : wildfire
coronanahh : coronanahh
what : what
a : a
good : good
way : way
start : start
year : year
henrijova : henrijova
yanwarga : yanwarga
jakardah : jakardah
punya : punya
tiang : tiang
toa : toa
peringatan : ingat
nwkwkwkkknuddudeuddudenntelan : nwkwkwkkknuddudeuddudenntelan
alat : alat
peru : peru
rofiiiiah : rofiiiiah
rupiah : rupiah
menguat : kuat
jiwasraya : jiwasraya
pembentukan : bentuk
pansus : pansus
panja : panja
kpk : kpk
melemah : lemah
kantor : kantor
pdip : pdip
gagal : gagal
digeledah : geledah
dll : dll
flood : flood
gate : gate
stasiun : stasiun
mrt : mrt
httpstcoqhhykhlpv : http

bikinkocak : bikinkocak
cewek : cewek
rasanyanninacoronavirusalert : rasanyanninacoronavirusalert
snackboxgratis : snackboxgratis
grammys : grammys
lucuabisbiku : lucuabisbiku
yusufdumdum : yusufdumdum
kasih : kasih
basuki : basuki
kemenpu : kemenpu
keras : keras
menguras : uras
underpass : underpass
kemayorannnsaat : kemayorannnsaat
gabeneru : gabeneru
kemarin : kemarin
jam : jam
berharihari : berharihari
bilangu : bilangu
polisi : polisi
india : india
uddudennniesbebalsokhebat : uddudennniesbebalsokhebat
surut : surut
pencitrau : pencitrau
pelaksaan : laksa
masalahgiliran : masalahgiliran
jabat : jabat
ribet : ribet
sekdanyannnon : sekdanyannnon
jobkan : jobkan
sajatumannrevitalisasi : sajatumannrevitalisasi
izinemang : izinemang
e : e
mbah : mbah
euddudennniesbebalsokhebatnnhttpstcoqecsivzp : euddudennniesbebalsokhebatnnhttpstcoqecsivzp
ketahuannera : ketahuannera
pejabat : jabat
kbingungannnmakanya : kbingungannnmakanya
dprd : dprd
action : action
dong : dong
httpstcokjcswaot : htt

...

httpstcojubwynnpenjelasan : httpstcojubwynnpenjelasan
dimasukkin : dimasukkin
messaging : messaging
provu : provu
distancingu : distancingu
berdampakununcatatannajwa : berdampakununcatatannajwa
rumahnnshare : rumahnnshare
favoritmunneh : favoritmunneh
httpstcogcgbespm : httpstcogcgbespm
indonesianhttpstcoomwzsixupnnvideo : indonesianhttpstcoomwzsixupnnvideo
ekarossirahmawt : ekarossirahmawt
mutiaraanberty : mutiaraanberty
aznlova : aznlova
bertebarannnuntuk : bertebarannnuntuk
pembekalan : bekal
hoaksnnbersama : hoaksnnbersama
kalahnnbawaslu : kalahnnbawaslu
menindaklanjutinya : menindaklanjutinya
pengkajiannnhttpstcorcrnyfakv : pengkajiannnhttpstcorcrnyfakv
httpstcooppdzynhow : httpstcooppdzynhow
armenwenas : armenwenas
andreyandreyka : andreyandreyka
pubgmid : pubgmid
gta : gta
gamenya : gamenya
ucsurat : ucsurat
braile : braile
rusakud : rusakud
checker : checker
dimodifikasi : modifikasi
nnhttpstcomjmoslwvt : nnhttpstcomjmoslwvt
httpstcojormojvcd : httpstcojormojvcd
dpshtxyz : dpsh

ucfrustasi : ucfrustasi
paslonudnnmelalui : paslonudnnmelalui
prabowonndari : prabowonndari
httpstcooorwalji : httpstcooorwalji
kekuasaannn : kekuasaannn
nnhttpstcojgybwzinzl : nnhttpstcojgybwzinzl
schroder : schroder
unonn : unonn
indonesiannhttpstcoyeroipj : indonesiannhttpstcoyeroipj
nantinnhttpstcomeskkmfa : nantinnhttpstcomeskkmfa
httpstcohypqjuiknl : httpstcohypqjuiknl
adajerawat : adajerawat
wargawarganya : wargawarganya
danielmarzuqi : danielmarzuqi
syaprud : syaprud
korbanin : korbanin
salahmenkes : salahmenkes
dwitunggal : dwitunggal
luhutjokowi : luhutjokowi
querencha : querencha
shaun : shaun
sheep : sheep
kokoakp : kokoakp
yahsonna : yahsonna
bestfriendny : bestfriendny
httpstcowejurpyw : httpstcowejurpyw
luhutnhttpstcovadbocxnn : luhutnhttpstcovadbocxnn
lututnya : lutut
gemetar : gemetar
satunnhttpstcoipjzgoubo : satunnhttpstcoipjzgoubo
aieks : aieks
waspadanjgn : waspadanjgn
jubirnbukan : jubirnbukan
lbpnnjgn : lbpnnjgn
percayancape : percayancape
dibohonginnappr : diboh

finlandia : finlandia
dibia : dibia
httpstcocuolnmothr : httpstcocuolnmothr
httpstcoepxicygzun : httpstcoepxicygzun
httpstcodkjmloh : httpstcodkjmloh
covidnnterus : covidnnterus
httpstcoeuaxxgxid : httpstcoeuaxxgxid
httpstconfwijuzs : httpstconfwijuzs
weallstandwithsaiddidunpernyataan : weallstandwithsaiddidunpernyataan
httpstcoyznqjju : httpstcoyznqjju
kasad : kasad
httpstcorrnjursn : httpstcorrnjursn
httpstcoxxwybrfqo : httpstcoxxwybrfqo
jayajd : jayajd
httpstcofthxbil : httpstcofthxbil
sayedesje : sayedesje
rizqen : rizqen
brat : brat
tat : tat
lagot : lagot
kebab : kebab
bbibbhi : bbibbhi
haniinr : haniinr
songood : songood
jipeeer : jipeeer
httpstcoyfoavpek : httpstcoyfoavpek
httpstcojhnjvlptci : httpstcojhnjvlptci
putriimay : putriimay
panjaitannsudah : panjaitannsudah
covidnnsaya : covidnnsaya
capek : capek
httpstcodzcjeidodw : httpstcodzcjeidodw
talioedeng : talioedeng
gelud : gelud
konorasah : konorasah
kesuwennsya : kesuwennsya
gininnek : gininnek
kwe : kwe
geeeeessnhttps : g

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

C:\Users\yunus\Anaconda3\envs\ROOT\lib\site-packages\tqdm\std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel



0        [lisaamartatara, fadlizon, bangga, bangga, buz...
1        [rustamibrahim, polling, gubernur, jawa, ridwa...
2        [jangan, sampai, anies, baswedan, nikmat, hasi...
3        [ghanieierfan, temu, haru, aniesbaswedan, tsun...
4        [sebastianteza, jangan, nympek, anies, basweda...
                               ...                        
50641    [mataniari, afaik, pardon, presiden, bentuk, a...
50642    [msaiddidu, sayang, luhut, paham, otak, uang, ...
50643    [rapat, kemenlu, anggota, dpr, ingat, recok, m...
50644    [hikosy, bekas, beda, galaksi, sana, dengar, d...
50645    [mending, rakyat, ramai, ancam, luhut, negara,...
Name: tweet_tokens_stemmed, Length: 50646, dtype: object


In [ ]:
TWEET_DATA.to_csv("Text_Preprocessing.csv")

In [ ]:
TWEET_DATA.to_excel("Text_Preprocessing.xlsx")

In [ ]:
TWEET_DATA.to_hdf("Text_Preprocessing.h5")